# Projeto Robo Seguidor de Linha usando OpenCV
## Type Robot: Robo OBR Resgate
### Comunidade: BigRobot  
#### Projetistas: Debora, Juan e Daniel


### Sumario 

1. Instalação do Ubuntu Mate
2. Instalação e configuração do OpenCV 
3. Instalação do Arduino
4. Programando OpenCV usando Simulação
5. Circuito eletronico do Robo
6. Programando no arduino
7. Topologia da rede
8. Usando SSH para acesso remoto no robo




## Instalanção do Ubuntu Mate

##  Instalação e configuração do OpenCV

## 